In [3]:
import spatialmath as sm
import spatialgeometry as sg
import roboticstoolbox as rtb
from pydrake.solvers import MathematicalProgram, Solve
import swift
import time
import numpy as np


def jacobian_i_k_optimisation(robot, v, v_max=1.2):
    # jacobian inverse kinematics with optimisation
    J = robot.jacobe(robot.q)
    prog = MathematicalProgram()
    v_opt = prog.NewContinuousVariables(6, "v_opt")
    # Define the error term for the cost function
    error = J @ v_opt - v
    prog.AddCost(error.dot(error))
    # Add bounding box constraint for joint velocities
    lower_bounds = [-v_max] * 6  # Lower bounds for each joint velocity
    upper_bounds = [v_max] * 6   # Upper bounds for each joint velocity
    prog.AddBoundingBoxConstraint(lower_bounds, upper_bounds, v_opt)
    # Solve the optimization problem
    result = Solve(prog)
    return result.is_success(), result.GetSolution(v_opt)

def end_effector_base_position_from_tip(T_tip ,offset):
    """
    Calculate the end effector base position given the tip position and an arbitrary offset.
    
    :param tip_position: A tuple of (x, y, z) coordinates for the end effector tip.
    :param tip_orientation: A tuple of (roll, pitch, yaw) angles in degrees for the end effector tip orientation.
    :param offset: A tuple of (x_e, y_e, z_e, roll_e, pitch_e, yaw_e) representing the offset of the base from the tip.
    :return: The position of the end effector base as a tuple (x, y, z).
    """
    x_e, y_e, z_e, roll_e, pitch_e, yaw_e = offset
    T_offset = sm.SE3.RPY([roll_e, pitch_e, yaw_e], unit='deg') * sm.SE3(x_e, y_e, z_e)
    
    # Calculate the transformation matrix for the base
    T_base = T_tip * T_offset.inv()
    return T_base

def generate_circular_trajectory(T0, r, n_points=100):
    """
    Generates a circular trajectory in 3D space.
    
    :param T0: Initial pose as an SE3 object.
    :param r: Radius of the circle.
    :param n_points: Number of points to generate along the circle.
    :return: A list of SE3 objects representing poses along the circular trajectory.
    """
    
    # Angle increments for generating points on the circle
    t = np.linspace(0, 1,  n_points, endpoint=False)
    theta = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    
    # Circle parameterization in its local XY plane
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    z = np.zeros_like(x)  # Assuming the circle lies in the XY plane
    
    # Generate poses along the circular trajectory
    trajectory = []
    for i, (xi, yi, zi) in enumerate(zip(x, y, z)):
        # Construct the translation vector in the local frame of T0
        local_translation = sm.SE3(xi, yi, zi)
        
        # Transform to the global frame using T0
        global_translation = T0 * local_translation

        # Append the global pose to the trajectory
        trajectory.append(global_translation)
    #return rtb.tools.trajectory.Trajectory("circle", trajectory, t)
    return trajectory

def ik_trajectory(robot, ctraj):
    q0 = robot.q
    qtraj = [robot.ikine_LM(c, mask=[1, 1, 1, 0.1, 0.1, 0.1], q0=q0).q for c in ctraj]


In [1]:
class Lite6:
    def __init__(self, simulation, robot_ip = None, tcp_offset = None) -> None:
        self.virtual_robot = rtb.models.URDF.Lite6()
        self.simulation = simulation
        if self.simulation:
            self.simulation.add(self.virtual_robot, robot_alpha=True, collision_alpha=False)
        self.tcp_offset = tcp_offset
        self.real_robot = None
        if robot_ip:
            from xarm.wrapper import XArmAPI
            from robot import RobotMain
            self.real_robot = RobotMain(XArmAPI(robot_ip, baud_checkset=False))
            print(self.real_robot)

    def move_to(self, dest, dt=0.05, gain=1, treshold=0.01, offset=True):
        if self.tcp_offset and offset:
            dest = end_effector_base_position_from_tip(dest, self.tcp_offset)
        if self.simulation:
            axes = sg.Axes(length=0.1, pose=dest)
            self.simulation.add(axes)
        if self.real_robot:
            self.real_robot._arm.set_mode(4)
            self.real_robot._arm.set_state(0)
        arrived = False
        while not arrived:
            if self.real_robot:
                q = self.real_robot._arm.get_servo_angle(is_radian=True)[1][:6]
                print(q)
                self.virtual_robot.q = q
            else:
                q = self.virtual_robot.q
            v, arrived = rtb.p_servo(self.virtual_robot.fkine(q), dest, gain=gain, threshold=treshold)
            qd = jacobian_i_k_optimisation(self.virtual_robot, v, v_max=1)[1]
            self.virtual_robot.qd = qd
            if self.real_robot:
                self.real_robot._arm.vc_set_joint_velocity(qd, is_radian=True)
            if self.simulation:
                self.simulation.step(dt)
        if self.real_robot:
            self.real_robot._arm.vc_set_joint_velocity([0, 0, 0, 0, 0, 0], is_radian=True)
        return arrived, self.virtual_robot.q

    def move_line(self, q0, T1, duration=1, n_samples=1000):
        resolution = duration/n_samples
        t = np.arange(0, duration, resolution)
        T0 = self.virtual_robot.fkine(q0)
        if self.tcp_offset:
            T1 = end_effector_base_position_from_tip(T1, lite6.tcp_offset)
        if self.real_robot:
            self.real_robot._arm.set_mode(1)
            self.real_robot._arm.set_state(0)
        Ts = rtb.ctraj(T0, T1, t)
        Js = self.virtual_robot.ikine_LM(Ts, mask=[1, 1, 1, 0.1, 0.1, 0.1], q0=q0)
        for q in Js.q:
            if self.simulation:
                self.virtual_robot.q = q
                self.simulation.step(resolution)
            if self.real_robot:
                self.real_robot._arm.set_servo_angle_j(q, is_radian=True)
                time.sleep(resolution)
        return True

    def execute_joint_trajectory(self, q_traj, duration):
        n_samples = len(q_traj)
        resolution = duration/n_samples
        if self.real_robot:
            self.real_robot._arm.set_mode(1)
            self.real_robot._arm.set_state(0)
        for q in q_traj:
            if self.simulation:
                self.virtual_robot.q = q
                self.simulation.step(resolution)
            if self.real_robot:
                self.real_robot._arm.set_servo_angle_j(q, is_radian=True)
                time.sleep(resolution)

    def move_circle(self, q0, r, duration=1, n_samples=1000):
        resolution = duration/n_samples
        t = np.arange(0, duration, resolution)
        T0 = self.virtual_robot.fkine(q0)
        traj = generate_circular_trajectory(T0, r)
        if self.real_robot:
            self.real_robot._arm.set_mode(1)
            self.real_robot._arm.set_state(0)
        for to in traj:
            q = lite6.virtual_robot.ikine_LM(to, mask=[1, 1, 1, 0.1, 0.1, 0.1], q0=q0).q
            if self.simulation:
                self.virtual_robot.q = q
                self.simulation.step(resolution)
            if self.real_robot:
                self.real_robot._arm.set_servo_angle_j(q, is_radian=True)
                time.sleep(resolution)

    
    def get_pose(self):
        return self.virtual_robot.fkine(self.virtual_robot.q)
    
    def reset(self):
        if self.simulation:
            self.virtual_robot.q = self.virtual_robot.qz
            self.simulation.step(0.1)
        if self.real_robot:
            self.real_robot._reset()
        


tcp_offset = [0, 0, 0.2, 0, 0, 0]
sim = swift.Swift()
sim.launch(realtime=True)
lite6 = Lite6(simulation=sim, robot_ip="192.168.1.159",  tcp_offset=tcp_offset)
#lite6 = Lite6(simulation=sim,  tcp_offset=tcp_offset)

NameError: name 'swift' is not defined

In [68]:

lite6.reset()
T0 = sm.SE3(0.95, 0.95, 0)*sm.SE3.RPY([-180, -180, 0], order='xyz', unit='deg')
T1 = sm.SE3(0.1, 0, 0)*sm.SE3.RPY([-180, -180, 0], order='xyz', unit='deg')
print(T0)

T0_E = end_effector_base_position_from_tip(T0, lite6.tcp_offset)
traj = generate_circular_trajectory(T0_E, 0.01)
q = []
_, q0 = lite6.move_to(traj[0], offset=False)
for to in traj:
    q.append(lite6.virtual_robot.ikine_LM(to, mask=[1, 1, 1, 1, 1, 1], q0=q0).q)


lite6.execute_joint_trajectory(q, 1)

   1         0         0         0.95      
   0        -1         0         0.95      
   0         0        -1         0         
   0         0         0         1         



connection handler failed
Traceback (most recent call last):
  File "/home/mrcyme/.local/lib/python3.10/site-packages/websockets/legacy/server.py", line 240, in handler
    await self.ws_handler(self)
  File "/home/mrcyme/.local/lib/python3.10/site-packages/websockets/legacy/server.py", line 1186, in _ws_handler
    return await cast(
  File "/home/mrcyme/miniconda3/envs/robotics-toolbox/lib/python3.10/site-packages/swift/SwiftRoute.py", line 320, in serve
    await self.expect_message(websocket, expected)
  File "/home/mrcyme/miniconda3/envs/robotics-toolbox/lib/python3.10/site-packages/swift/SwiftRoute.py", line 325, in expect_message
    recieved = await websocket.recv()
  File "/home/mrcyme/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 568, in recv
    await self.ensure_open()
  File "/home/mrcyme/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 935, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.Connec

KeyboardInterrupt: 